Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [1]:
import pandas as pd
wage = pd.read_csv('wage.csv')

3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [2]:
wage.loc[(wage.gender == 0), 'gender'] = 'F'
wage.loc[(wage.gender == 1), 'gender'] = 'M'

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [3]:
wage.groupby('gender')['wage'].mean()

gender
F    40855.747261
M    46815.944005
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [4]:
dubli = wage.person_id.value_counts()
wage.loc[dubli[dubli > 1].index, 'wage']

17     13434.655493
16     13434.655493
15    273483.609236
14    273483.609236
13     23547.663281
12     23547.663281
Name: wage, dtype: float64

In [5]:
wage = wage.drop_duplicates()

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [6]:
wage.describe()

,person_id,wage
count,1000.000000,1000.000000
mean,499.500000,43694.227404
std,288.819436,55352.539343
min,0.000000,-287418.645743
25%,249.750000,14489.682367
50%,499.500000,27309.529498
75%,749.250000,52021.080258
max,999.000000,755320.874132


In [7]:
wage = wage[wage['wage'] > 0]

In [8]:
wage.describe()

,person_id,wage
count,995.000000,995.000000
mean,501.844221,44306.969585
std,287.638377,54302.194392
min,0.000000,945.648458
25%,253.500000,14683.306148
50%,502.000000,27519.361794
75%,750.500000,52267.313664
max,999.000000,755320.874132


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [9]:
bonus = pd.read_csv('bonus.csv', sep=';')

8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [10]:
df = wage.merge(bonus, how='outer')

9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [11]:
df['bonus'].fillna(value = 0, inplace = True)
df = df[df['wage'] > 0]
df = df.assign(total = df.wage * 12 + df.bonus)
df[['gender', 'wage']].groupby('gender').agg(['mean', 'median'])

wage              
                mean        median
gender                            
F       41353.986687  26721.001610
M       47081.694269  29006.633828

10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [12]:
df.to_csv('new_df_wage.csv', index=False)